In [1]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [2]:
data = pd.read_csv("Churn_Modelling.csv")
data.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [3]:
# Preprocessing data
data.drop(columns=["RowNumber", "CustomerId", "Surname"],axis=1,inplace=True)
data.head(3)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [4]:
# Encoding Categorical Variables
encoder_label_gender = LabelEncoder()
data['Gender'] = encoder_label_gender.fit_transform(data['Gender'])

In [5]:
# One Hot Encoding
encoder_onehot_geography = OneHotEncoder()
geo_encoder = encoder_onehot_geography.fit_transform(data[['Geography']])
geo_encoder 

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [6]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns=encoder_onehot_geography.get_feature_names_out(['Geography']))

In [7]:
# Combine one hot encoder columns with the original dataframe
data = pd.concat([data.drop('Geography', axis=1),geo_encoded_df],axis=1)
data.head(2)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0


In [8]:
# Save the Encoders
with open('labelencoder_gender.pkl','wb') as file:
    pickle.dump(encoder_label_gender, file)

with open('OneHotEncoder_Geography', 'wb') as file:
    pickle.dump(encoder_onehot_geography, file)

In [9]:
# Dividing the dataset into independent and dependent features
X = data.drop(columns=['Exited'], axis=1)
y = data['Exited']

# Split data into training and testing 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [10]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# saving scaler
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

## ANN Implementation

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [12]:
# Build our ANN model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), # HL1 connected with input layer
    Dense(32, activation='relu'), # HL2
    Dense(1, activation='sigmoid') # output layer
])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [15]:
# compile the model
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
# set up tensor board
log_dir = "logs/fit/" +datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [17]:
# set up early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=5, restore_best_weights=True)

In [18]:
# Training the model
history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=100,
    callbacks=[tensorflow_callback, early_stopping_callback]
)

Epoch 1/100


219/219 [==============================] - 2s 4ms/step - loss: 0.3959 - accuracy: 0.8347 - val_loss: 0.3456 - val_accuracy: 0.8580
Epoch 2/100
219/219 [==============================] - 1s 3ms/step - loss: 0.3551 - accuracy: 0.8507 - val_loss: 0.3401 - val_accuracy: 0.8657
Epoch 3/100
219/219 [==============================] - 1s 3ms/step - loss: 0.3548 - accuracy: 0.8540 - val_loss: 0.3496 - val_accuracy: 0.8590
Epoch 4/100
219/219 [==============================] - 1s 3ms/step - loss: 0.3499 - accuracy: 0.8551 - val_loss: 0.3338 - val_accuracy: 0.8660
Epoch 5/100
219/219 [==============================] - 1s 3ms/step - loss: 0.3464 - accuracy: 0.8540 - val_loss: 0.3393 - val_accuracy: 0.8617
Epoch 6/100
219/219 [==============================] - 1s 3ms/step - loss: 0.3397 - accuracy: 0.8611 - val_loss: 0.3370 - val_accuracy: 0.8660
Epoch 7/100
219/219 [==============================] - 1s 3ms/step - loss: 0.3396 - accuracy: 0.8586 - val_loss: 0.3521 - val_accuracy: 0.86

In [19]:
# saving the model
model.save('model.h5')
# h5 file is compatible with keras that is why we are saving it in h5 format

c:\projects\ANN_Classification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
# Launch tensorboard extension
%load_ext tensorboard

In [27]:
%tensorboard --logdir logs/fit20250120-141452

Reusing TensorBoard on port 6006 (pid 21416), started 1 day, 23:57:30 ago. (Use '!kill 21416' to kill it.)